# Create TensorFlow Session

In [1]:
import tensorflow as tf
from keras import backend as K

sess = tf.Session()
K.set_session(sess)

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


# Define Model

In [2]:
from keras.models import Model
from keras.layers import Input, RepeatVector, TimeDistributed, Flatten, SimpleRNN, Reshape
from keras.layers.merge import Multiply
from keras.layers import Conv2D, Permute, MaxPooling1D, Activation
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.constraints import nonneg

T, H, W, C, D = 2, 28, 28, 1, 10

img = Input(shape=[H, W*T, C], name='Image')

with tf.name_scope('Repeat'):
    flat_img = Flatten()(img)
    flat_imgs = RepeatVector(n=T)(flat_img)

imgs = Reshape([T, H, W*T, C], name='Images')(flat_imgs)
flat_masks = SimpleRNN(units=H*W*T, activation='sigmoid', return_sequences=True)(flat_imgs)
masks = Reshape([T, H, W*T, C], name='Masks')(flat_masks)
masked_imgs = Multiply()([imgs, masks])

with tf.name_scope('Conv'):
    matched = TimeDistributed(Conv2D(filters=D,
                                     kernel_size=[H, W],
                                     strides=[1, W],
                                     kernel_constraint=nonneg()))(masked_imgs)
    matched = Reshape([T, T, D])(matched)
    
with tf.name_scope('Pool'):
    pooled = TimeDistributed(MaxPooling1D(pool_size=T))(matched)
    scores = Reshape([T, D])(pooled)

probs = TimeDistributed(Activation('softmax'), name='Probs')(scores)

model = Model(inputs=img, outputs=probs)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
Image (InputLayer)               (None, 28, 56, 1)     0                                            
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 1568)          0                                            
____________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)   (None, 2, 1568)       0                                            
____________________________________________________________________________________________________
simple_rnn_1 (SimpleRNN)         (None, 2, 1568)       4918816                                      
___________________________________________________________________________________________

# Visualize Model

1. TensorBoard
2. Keras Layers

In [3]:
from support import show_graph

show_graph(sess.graph)
# SVG(model_to_dot(model).create(prog='dot', format='svg'))

# Load MNIST Digits

In [4]:
from IPython.display import display
from keras.preprocessing.image import array_to_img
import numpy as np
from keras.datasets import mnist

[X_mnist, y_mnist], _ = mnist.load_data()
X_mnist = X_mnist/255.

digit_idxs = [np.argwhere(y_mnist == d).flatten()[0] for d in range(10)]
X_mnist = X_mnist[digit_idxs]
for x in X_mnist:
    img_ = array_to_img(np.expand_dims(x, axis=0), data_format='channels_first')
    display(img_)

X_mnist.shape

(10, 28, 28)

# Create Images and Labels

In [5]:
import itertools
from keras.preprocessing.image import array_to_img

nb_train = 90
X, y = np.zeros([nb_train, H, W*T]), [None]*nb_train
i = 0
D = set()
while i < nb_train:
    digits = np.random.choice(10, size=T, replace=False)
    if tuple(digits) in D:
        continue
        
    D.add(tuple(digits))
    X[i] = np.hstack([x for x in X_mnist[digits]])
    y[i] = digits
    i += 1

for idx in range(5):
    x = X[idx]
    img_ = array_to_img(np.expand_dims(X[idx], axis=-1))
    print(y[idx])
    display(img_)

[0 7]


[9 0]


[2 5]


[0 3]


[0 9]


# Make Labels Categorical

In [6]:
Y = np.zeros([nb_train, T, 10])
for i, ys in enumerate(y):
    for j, y_ in enumerate(ys):
        Y[i, j, y_] = 1
    
Y.shape

(90, 2, 10)

# Fit Model

In [7]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

X_ = np.expand_dims(X, axis=-1) # add color channel
model.fit(X_, Y, epochs=50, validation_split=.1)

Train on 81 samples, validate on 9 samples
Epoch 1/50


KeyboardInterrupt: 

# Visualize Convolutional Filters

In [9]:
W, b = model.layers[7].get_weights()

for i in range(10):
    w = W[..., i]
    img_ = array_to_img(w)
    display(img_)

# Get Masks and Masked Images

In [10]:
model_ = Model(inputs=img, outputs=[imgs, masks, masked_imgs])
model_.reset_states()
I, M, MI = model_.predict(np.expand_dims(X, axis=-1))

# Visualize Masks and Masked Images

In [11]:
idx = np.random.choice(nb_train)

for i, m, mi in zip(I[idx], M[idx], MI[idx]):
    img, mask, masked_img = array_to_img(i), array_to_img(m), array_to_img(mi)
    display(img)
    display(mask)
    display(masked_img)
    print('---')

---


---
